In [11]:
import numpy as np
import pandas as pd
from numpy import linalg as LA

#### System user i

In [32]:
def system_user_i(Q, M, Lambda, i):
    
#     Find the number of features from M
    f = np.size(M,0);
    
#     Determine the number of elements inthe index set (the number of movies that have been ranked by user i)
    mJi = np.count_nonzero(Q[:,i]);
    
#     Determine the index set with all users that have ranked movie j
    Ji = np.argwhere(Q[:,i]).reshape(mJi);
        
#     The submatrix of U containing all the columns of U corresponding to users who have ranked movie j
    M_Ji = M[:,Ji];

#     The ratings given to movie j
    q_Ji = Q[Ji,i];

#     The matrix in the system for computing the optimal m_j
    Ai = np.matmul(M_Ji, M_Ji.T) + Lambda*mJi*np.eye(f);

#     The RHS in the system for computing the optimal m_j
    bi = np.matmul(M_Ji, q_Ji);
    
    return [Ai, bi]

#### System movie j

In [33]:
def system_movie_j(R, U, Lambda, j):
    
#     Find the number of features from U
    f = np.size(U,0);
    
#     Determine the number of elements inthe index set (the number of users who have ranke movie j)
    nIj = np.count_nonzero(R[:,j]);    
    
#     Determine the index set with all users that have ranked movie j
    Ij = np.argwhere(R[:,j]).reshape(nIj);
    
#     The submatrix of U containing all the columns of U corresponding to users who have ranked movie j
    U_Ij = U[:,Ij];

#     The ratings given to movie j
    r_Ij = R[Ij,j];

#     The matrix in the system for computing the optimal m_j
    Aj = np.matmul(U_Ij, U_Ij.T) + Lambda*nIj*np.eye(f);

#     The RHS in the system for computing the optimal m_j
    bj = np.matmul(U_Ij, r_Ij);
    
    return [Aj, bj]

#### One iteration of ALS

In [34]:
def myALS(R, U, M, Lambda):

#     Defining number of movies (m) and users (n):
    n = np.size(R, 1);
    m = np.size(R, 0);
    
#     Going through every column solviung for matrix M
    for j in range(n):
        [Aj, bj] = system_movie_j(R,U,Lambda,j);
        M[:, j] = np.linalg.solve(Aj, bj)
    
#     Going through every column solving for matrix U, part that doesn't work
    for i in range(m):
        [Aj, bj] = system_user_i(R.T,M,Lambda,i);
        U[:, i] = np.linalg.solve(Aj, bj)
    return [U, M]

#### gValue (The function g we are optimising)

In [36]:
def gValue(R,U,M,Lambda):

    n = np.size(R, 1);
    m = np.size(R, 0);
    Q = R.T;

    a=0;
    b=0;
    for j in range(n):
        nIj = np.count_nonzero(R[:,j]);
        Ij = np.argwhere(R[:,j]).reshape(nIj);
        
        a = a + LA.norm(R[Ij,j]-np.matmul(U[:,Ij],M[:,j]))**2;

        b = b + nIj*LA.norm(M[:,j])**2;

    c=0;
    for i in range(m):
        mJi = np.count_nonzero(Q[:,i]);
        Ji = np.argwhere(Q[:,i]).reshape(mJi);
    
        c = c + mJi*LA.norm(U[:,i])**2;
        
    g = a + Lambda*(c+b);
    return g

#### Test

In [37]:
# Initialising
Lambda=0.01;
nits=500;
f=2;

R = np.array([[5, 0, 0], [0, 4, 0], [4, 5, 1], [0, 0, 2], [0, 2, 0]]);
U=np.array([[1., 2., 3., 4., 5.],[ 1., 2., 3., 4., 5.]]);
M=np.ones(f*3).reshape((f,3))


for i in range(nits):
    [U, M] = myALS(R,U,M,Lambda)
    R_pred = np.matmul(U.T, M)

In [38]:
R_pred

array([[4.98605527, 6.20091582, 1.24553568],
       [3.21052881, 3.9927801 , 0.80276011],
       [3.99922464, 4.97364315, 0.9999986 ],
       [1.2631687 , 1.57242606, 1.98585785],
       [1.60526441, 1.99639005, 0.40138006]])

### For 0 included and unknown values as  NaN values  Does not work as well due to inherently ALS is defined for sparse matrixes

##### System Movies _ j

In [19]:
def system_movie_j(R, U, Lambda, j):
    
#     Find the number of features from U
    f = np.size(U,0);
    
#     Determine the number of elements inthe index set (the number of users who have ranke movie j)
    nIj = np.count_nonzero(np.isnan(R[:,j]) == False);    
    
#     Determine the index set with all users that have ranked movie j
    Ij = np.argwhere(np.isnan(R[:,j]) == False).reshape(nIj);
    
#     The submatrix of U containing all the columns of U corresponding to users who have ranked movie j
    U_Ij = U[:,Ij];

#     The ratings given to movie j
    r_Ij = R[Ij,j];

#     The matrix in the system for computing the optimal m_j
    Aj = np.matmul(U_Ij, U_Ij.T) + Lambda*nIj*np.eye(f);

#     The RHS in the system for computing the optimal m_j
    bj = np.matmul(U_Ij, r_Ij);
    
    return [Aj, bj]

##### System User_i 

In [20]:
def system_user_j(Q, M, Lambda, i):
    
#     Find the number of features from U
    f = np.size(M,0);
    
#     Determine the number of elements inthe index set (the number of users who have ranke movie j)
    mJi = np.count_nonzero(np.isnan(Q[:,i]) == False);
    
    
#     Determine the index set with all users that have ranked movie j
    Ji = np.argwhere(np.isnan(Q[:,i]) == False).reshape(mJi);
        
#     The submatrix of U containing all the columns of U corresponding to users who have ranked movie j
    M_Ji = M[:,Ji];

#     The ratings given to movie j
    q_Ji = Q[Ji,i];

#     The matrix in the system for computing the optimal m_j
    Ai = np.matmul(M_Ji, M_Ji.T) + Lambda*mJi*np.eye(f);

#     The RHS in the system for computing the optimal m_j
    bi = np.matmul(M_Ji, q_Ji);
    
    return [Ai, bi]

##### One iteration of the ALS algorithm

In [21]:
def myALS(R, U, M, Lambda):

#     Defining number of movies (m) and users (n):
    n = np.size(R, 1);
    m = np.size(R, 0);
    
#     Going through every column solviung for matrix M
    for j in range(n):
        [Aj, bj] = system_movie_j(R,U,Lambda,j);
        M[:, j] = np.linalg.solve(Aj, bj)
    
#     Going through every column solving for matrix U
    for i in range(m):
        [Ai, bi] = system_movie_j(R.T,U,Lambda,j);
        U[:, j] = np.linalg.solve(Ai, bi)
    return [U, M]

##### gValue (Function we optimise, not that important to the results)

In [49]:
def gValue(R,U,M,Lambda):

    n = np.size(R, 1);
    m = np.size(R, 0);
    Q = R.T;

    a=0;
    b=0;
    for j in range(n):
        nIj = np.count_nonzero(np.isnan(R[:,j]) == False);
        Ij = np.argwhere(np.isnan(R[:,j]) == False).reshape(nIj);
        
        a = a + LA.norm(R[Ij,j]-np.matmul(U[:,Ij],M[:,j]))**2;

        b = b + nIj*LA.norm(M[:,j])**2;

    c=0;
    for i in range(m):
        mJi = np.count_nonzero(np.isnan(Q[:,i]) == False);
        Ji = np.argwhere(np.isnan(Q[:,i]) == False).reshape(mJi);
    
        c = c + mJi*LA.norm(U[:,i])**2;
        
    g = a + Lambda*(c+b);
    return g

##### Testing parameters

In [25]:
# Initialising
Lambda=0.01;
nits=2;
f=2;

R = np.array([[5, np.nan, np.nan], [np.nan, 4, np.nan], [4, 5, 1], [np.nan, np.nan, 2], [np.nan, 2, np.nan]]);
U=np.array([[1., 2., 3., 4., 5.],[ 1., 2., 3., 4., 5.]]);
M=np.ones(f*3).reshape((f,3))


for i in range(nits):
    [U, M] = myALS(R,U,M,Lambda)
    R_pred = np.matmul(U.T, M)

In [26]:
R_pred

array([[ 4.05689684,  0.78550265,  0.52757942],
       [ 8.11379368,  1.57100529,  1.05515883],
       [ 5.63671949,  1.09136528,  0.73302336],
       [16.22758736,  3.14201059,  2.11031767],
       [20.2844842 ,  3.92751324,  2.63789708]])